In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
import shap
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

In [ ]:
# Using standard naming convention will improve the understandability of the notebook and "human" learning. 

train_data = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv',index_col=0)
test_data  = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv', index_col=0)
sample     = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
base_features = ['f52','f77','f81','f13']

X_full = train_data.copy()
y_full = X_full.pop('loss')

train_X, val_X, train_y, val_y = train_test_split(X_full, y_full,random_state = 0) #Default is 33.33% split, which is verified below

In [ ]:
train_data.head()

In [ ]:
train_dataset = cb.Pool(train_X, train_y) 
test_dataset = cb.Pool(val_X, val_y)

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE')

In [ ]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
model.grid_search(grid, train_dataset)

In [ ]:
pred = model.predict(val_X)
rmse = (np.sqrt(mean_squared_error(val_y, pred)))
r2 = r2_score(val_y, pred)
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

In [ ]:
sorted_feature_importance = model.feature_importances_.argsort()

In [ ]:
model.get_all_params()

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(val_X)
shap.summary_plot(shap_values, val_X)

In [ ]:
#Going for submission
pred = model.predict(test_data)

In [ ]:
plt.hist(pred)

In [ ]:
my_submission = pd.DataFrame({'id': test_data.index, 'loss':pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)